In [1]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score , StratifiedKFold

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

/home/kirill/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def validate(x , y):
    model = XGBClassifier(max_depth = 10 , n_estimators=670 , learning_rate=0.09 , colsample_bytree=0.9 , colsample_bylevel=0.6)
    cv = StratifiedKFold(4 ,shuffle=True, random_state=99)
    score = cross_val_score(model , x , y , scoring='roc_auc' , cv=cv)
    print (score.mean() , score.std() , '\n')

### Загружаем данные

In [3]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
all_data = pd.concat([train , test])

sample = pd.read_csv('data/sample_submission.csv')

### tf_idf

In [4]:
bigram_vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(1,10), max_features = 100 , encoding='utf8')

for_tf_idf = bigram_vectorizer.fit_transform(all_data.Word.apply(str.lower).tolist())

tfidf_sparse = TfidfTransformer().fit_transform(for_tf_idf)

In [5]:
tfidf = pd.DataFrame(tfidf_sparse.toarray() , index=all_data.index, columns=['tf_idf_' + str(i) for i in range(tfidf_sparse.shape[1])])

### cross-val-score

In [6]:
validate(tfidf.iloc[train.index] , train['Label'])

0.848157337699 0.00473124614843 



In [ ]:
model = XGBClassifier(max_depth = 10 , n_estimators=670 , learning_rate=0.09 , colsample_bytree=0.9 , colsample_bylevel=0.6)
model.fit(tfidf.iloc[train.index] , train['Label'])
sample['Prediction'] = model.predict_proba(tfidf.iloc[test.index] , axis=1)[:,0]
sample.to_csv('submit.csv' , index=False)